## Required functions
Run before executing the dataset pre-processing code.

In [ ]:
import os, glob, sys
from pycromanager import Dataset
from skimage import io, img_as_ubyte, exposure
import numpy as np

def thres_saturation(img, t=15, img_channel=3):
    img = img_as_ubyte(img)
    if img_channel==3:
        # typical t = 15
        img = rgb2hsv(img)
        h, w, c = img.shape
        sat_img = img[:, :, 1]
        sat_img = img_as_ubyte(sat_img)
        ave_sat = np.sum(sat_img) / (h * w)
    if img_channel==1:
        # typical t = 20
        h, w = img.shape
        ave_sat = np.sum(img) / (h * w)
    return ave_sat >= t

## Configruations
Run after the adjustment.

In [ ]:
data_config = {
    "resolution" : 256,
    "background-threshold" : 15
}

## Datasets pre-processing code

In [ ]:
data_folder = os.path.join('raw-data', str(data_config["resolution"]), '*')
datasets = glob.glob(data_folder)
slices_path = 'data'
os.makedirs(os.path.join('data', str(data_config["resolution"])), exist_ok=True)
for data_path in datasets:
    try:
        print(data_path)
        dataset = Dataset(data_path)
        for pos in range(0, 200):
            z_idx = 0
            while(dataset.has_image(position=pos, z=z_idx)):
                img = dataset.read_image(position=pos, z=z_idx)
                z_idx = z_idx+1
                if thres_saturation(img, t=data_config["background-threshold"], img_channel=1):
                    save_name = os.path.join(slices_path, data_path.split(os.sep)[-1].split(".")[0]+'_'+str(pos)+'_'+str(z_idx)+'.png')
                    io.imsave(save_name, img_as_ubyte(img))
    except:
        print("Dataset open failed! /n")